In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import string as s
%matplotlib inline

In [2]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.4260,39.0,165.0,3025654.0,3.010850e+09
5,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,9.884187e+05,219825.5227,3.0,145.0,1076229.0,1.011310e+09
6,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,9.883112e+05,219885.2785,3.0,145.0,1076229.0,1.011310e+09
7,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,9.877691e+05,217157.8561,3.0,133.0,1086093.0,1.010410e+09
8,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,9.630732e+05,156635.5542,NaN,NaN,NaN,NaN
9,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,9.926537e+05,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


In [3]:
for x in range(0, 5000, 1000):
    soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=1000&$offset=' + str(x) +\
        '&$select=boroname,spc_common,health,steward,count(tree_id)' +\
        '&$group=boroname,spc_common,health,steward').replace(' ', '%20')
    soql_trees = pd.read_json(soql_url)
    if(x==0):
        df = pd.DataFrame(columns=list(soql_trees.columns.values))
    df = df.append(soql_trees)

df = df.reset_index(drop=True)

In [4]:
df.head()


,boroname,spc_common,health,steward,count_tree_id
0,Bronx,sycamore maple,Good,3or4,1
1,Bronx,London planetree,Poor,None,175
2,Manhattan,Sophora,Good,3or4,484
3,Brooklyn,European alder,Fair,None,1
4,Queens,'Schubert' chokecherry,Good,4orMore,3


In [5]:
#Data for question 1:
df_totals = df.groupby(['boroname', 'spc_common'])['count_tree_id'].sum()
df_total_by_boroname_specie_health = df.groupby(['boroname', 'spc_common', 'health'])['count_tree_id'].sum()
df_totals = df_totals.reset_index(drop=False)
df_total_by_boroname_specie_health = df_total_by_boroname_specie_health.reset_index(drop=False)
df_totals.columns = ['boroname', 'spc_common', 'total_for_specie_in_borough']
df_total_by_boroname_specie_health.columns = ['boroname', 'spc_common', 'health', 'total']
tree_proportions = pd.merge(df_total_by_boroname_specie_health, df_totals, on=['boroname', 'spc_common'])
tree_proportions['ratio'] = tree_proportions['total']/ tree_proportions['total_for_specie_in_borough']
tree_proportions['spc_common'] = tree_proportions['spc_common'].apply(lambda x: x.title())


#Species dropdown:
species = np.sort(tree_proportions.spc_common.unique())

In [6]:
#Data for question 2:
df_total_by_steward = df.groupby(['boroname', 'spc_common', 'steward'])['count_tree_id'].sum()
df_total_by_steward = df_total_by_steward.reset_index(drop=False)
df_total_by_steward.columns = ['boroname', 'spc_common', 'steward', 'steward_total']
df_steward = pd.merge(df, df_total_by_steward, on=['boroname', 'spc_common', 'steward'])
di = {'Poor':1, 'Fair':2, 'Good':3}
df_steward['health_level'] = df_steward['health'].map(di)
df_steward['health_index'] = (df_steward['count_tree_id']/df_steward['steward_total']) * df_steward['health_level']
df_overall_health_index = df_steward.groupby(['boroname', 'spc_common', 'steward'])['health_index'].sum()
df_overall_health_index = df_overall_health_index.reset_index(drop=False)
df_overall_health_index.columns = ['boroname', 'spc_common', 'steward', 'overall_health_index']
di2 = {'3or4':3, '4orMore':4, 'None':1, '1or2':2}
df_overall_health_index['steward_level'] = df_overall_health_index['steward'].map(di2)
df_overall_health_index['spc_common'] = df_overall_health_index['spc_common'].apply(lambda x: x.title())
df_overall_health_index.head(10)

,boroname,spc_common,steward,overall_health_index,steward_level
0,Bronx,'Schubert' Chokecherry,1or2,2.823529,2
1,Bronx,'Schubert' Chokecherry,3or4,2.909091,3
2,Bronx,'Schubert' Chokecherry,4orMore,3.000000,4
3,Bronx,'Schubert' Chokecherry,None,2.702290,1
4,Bronx,American Beech,1or2,2.166667,2
5,Bronx,American Beech,3or4,2.500000,3
6,Bronx,American Beech,None,2.695652,1
7,Bronx,American Elm,1or2,2.696774,2
8,Bronx,American Elm,3or4,2.850000,3
9,Bronx,American Elm,4orMore,3.000000,4


In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets = external_stylesheets)


In [ ]:
app.layout = html.Div([
    html.H3('Select Tree Specie'),
    
    dcc.Dropdown(
        id='specie', 
        options=[{'label': i, 'value': i} for i in species],
        value="American Elm",
        style={'height': 'auto', 'width': '300px'}
    ),

    dcc.Graph(id='graph-ratio'),

    dcc.Graph(id='graph-health')

], style={'columnCount': 1})

#Display Proportion Graph 
@app.callback(
    Output('graph-ratio', 'figure'),
    [Input('specie', 'value')])
def update_figure(selected_specie):

    filtered_df = tree_proportions[tree_proportions.spc_common == selected_specie]
    manhattan = filtered_df[filtered_df.boroname == 'Manhattan']
    bronx = filtered_df[filtered_df.boroname == 'Bronx']
    brooklyn = filtered_df[filtered_df.boroname == 'Brooklyn']
    queens = filtered_df[filtered_df.boroname == 'Queens']
    staten_island = filtered_df[filtered_df.boroname == 'Staten Island']
    
    traces = []

    traces.append(go.Bar(
    x=queens['health'],
    y=queens['ratio'],
    name='Queens',
    opacity=0.85
    ))

    traces.append(go.Bar(
    x=manhattan['health'],
    y=manhattan['ratio'],
    name='Manhattan',
    opacity=0.85
    ))

    traces.append(go.Bar(
    x=bronx['health'],
    y=bronx['ratio'],
    name='Bronx',
    opacity=0.85
    ))

    traces.append(go.Bar(
    x=brooklyn['health'],
    y=brooklyn['ratio'],
    name='Brooklyn',
    opacity=0.85
    ))

    traces.append(go.Bar(
    x=staten_island['health'],
    y=staten_island['ratio'],
    name='Staten Island',
    opacity=0.9
    ))
    
    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'title': 'Health of Trees'},
            yaxis={'title': 'Proportion of Trees in Borough'},
            margin={'l': 40, 'b': 40, 't': 100, 'r': 10},
            legend=dict(x=-.1, y=1.2)
        )
    }


#Steward-Health Plot for Question 2
@app.callback(
    Output('graph-health', 'figure'),
    [Input('specie', 'value')])
def update_figure2(selected_specie):
    filtered_df = df_overall_health_index[df_overall_health_index.spc_common == selected_specie]
    traces2 = []
        
    for i in filtered_df.boroname.unique():
        df_by_borough = filtered_df[filtered_df['boroname'] == i]
        traces2.append(go.Scatter(
            x=df_by_borough['steward_level'],
            y=df_by_borough['overall_health_index'],
            mode='markers',
            opacity=0.7,
            marker={
                'size': 17,
                'line': {'width': 0.5, 'color': 'white'}
            },
            name=i
        ))
    
    return {
        'data': traces2,
        'layout': go.Layout(
            yaxis={'title': 'Overall Health Index'},
            xaxis=dict(tickvals = [1,2,3,4], ticktext = ['None', '1or2', '3or4', '4orMore'], title='Steward Level'),
            margin={'l': 40, 'b': 40, 't': 15, 'r': 10},
            legend=dict(x=-.1, y=1.2)
        )
    }


if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Oct/2020 19:04:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:04:11] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:04:11] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:04:11] "GET /_favicon.ico?v=1.16.3 HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:04:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:04:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:07:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:07:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:07:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:07:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:07:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020 19:07:46] "GET /_favicon.ico?v=1.16.3 HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2020